In [5]:
import pandas as pd

In [6]:
import pandas as pd
import re

# 1) your email‑cleaning function
def clean_email(email_str):
    if pd.isna(email_str) or not str(email_str).strip():
        return None
    email = str(email_str).strip().lower()
    email = re.sub(r'\s+', '', email)  # remove any internal whitespace
    pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
    return email if re.match(pattern, email) else None

# 2) load your sheet
file_path = 'Queens Office_Flex Broker Contact Info New.xlsx'
df = pd.read_excel(file_path)

# 3) find all columns with “email” in their name
email_cols = [c for c in df.columns if 'email' in c.lower()]

# 4) clean each one into a new parallel column
for col in email_cols:
    df[f'{col}_clean'] = df[col].apply(clean_email)

# 5) coalesce into a single Email column (first non‐null cleaned email)
clean_cols = [f'{c}_clean' for c in email_cols]
df['Email'] = df[clean_cols].bfill(axis=1).iloc[:, 0]

# 6) drop rows with no valid email
df = df[df['Email'].notna()].copy()

# 7) drop duplicates, keeping the first occurrence of each Email
df_unique = df.drop_duplicates(subset=['Email'])

# 8) (optional) remove the intermediate “_clean” columns
df_unique = df_unique.drop(columns=clean_cols)

# Now df_unique contains ALL your original columns + a cleaned 'Email',
# with each email appearing only once.
# You can export:
df_unique.to_csv('unique_contacts_full_new.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'Queens Office_Flex Broker Contact Info New.xlsx'

In [7]:
import pandas as pd
import os

def process_email_column(input_file_path, output_file_path):
    """
    Reads a CSV file, converts the 'Email' column to lowercase,
    and saves the result to a new CSV file.

    Args:
        input_file_path (str): The path to the input CSV file.
        output_file_path (str): The path where the output CSV file will be saved.
    """
    
    try:
        # Read the Excel file into a pandas DataFrame.
        df = pd.read_excel("Contacts All in One.xlsx")

        # Define the name of the email column.
        email_column = 'Email'

        # Check if the email column exists in the DataFrame.
        if email_column in df.columns:
            print(f"Found '{email_column}' column. Processing...")

            # Convert all entries in the 'Email' column to lowercase.
            # .fillna('') ensures that any empty cells (NaN values) are replaced with
            # an empty string before attempting to use the .str accessor.
            df[email_column] = df[email_column].fillna('').astype(str).str.lower()

            # Save the modified DataFrame to a new CSV file.
            # index=False prevents pandas from writing the DataFrame index as a column.
            df.to_csv(output_file_path, index=False)

            print(f"Successfully processed the file.")
            print(f"The updated data has been saved to: {output_file_path}")
        else:
            print(f"Error: Column '{email_column}' not found in the file.")
            print(f"Available columns are: {df.columns.tolist()}")

    except FileNotFoundError:
        print(f"Error: The file '{input_file_path}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- Main execution ---
if __name__ == "__main__":
    # Name of the input file you've uploaded.
    input_filename = 'Contacts All in One.xlsx - All Contacts- Stage 1.csv'

    # Define the name for the new output file.
    output_filename = 'contacts_emails_lowercased.csv'

    # Check if the input file exists in the current directory.
    if os.path.exists(input_filename):
        process_email_column(input_filename, output_filename)
    else:
        print(f"Error: Make sure the input file '{input_filename}' is in the same directory as the script.")

process_email_column("Contacts All in One.xlsx", "contacts_emails_lowercased.csv")

Error: Make sure the input file 'Contacts All in One.xlsx - All Contacts- Stage 1.csv' is in the same directory as the script.
Found 'Email' column. Processing...
Successfully processed the file.
The updated data has been saved to: contacts_emails_lowercased.csv


In [ ]:
full = pd.read_csv('contacts_emails_lowercased.csv')
full

,Property Address,Neighbor,Date,Name,Business Name/Type,Identity,Source,Assign To,Industry,Rent/Buy/Sell,...,Title,usage,SF MIN,SF MEAN,SF MAX,Budget,I,Location,Notes,Unnamed: 23
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2178,NaN,NaN,NaN,Steven Q,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
